### Requirements
- Weber County LIR Parcels (2019)
- TAZ Boundaries
- Owned Unit Groupings Boundaries
- Address points

In [1]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

## Inputs

In [2]:
taz_shp = '.\\Inputs\\TAZ.shp'
parcels = '.\\Inputs\\Utah_Weber_County_Parcels_LIR.gdb\\Parcels_Weber_LIR_UTM12'
address_pts = '.\\Inputs\\AddressPoints.gdb\\AddressPoints_Weber'
common_areas = '.\\Inputs\\Weber_Owned_Unit_Groupings.gdb\\Weber_Owned_Unit_Groupings'

## Create geodatabases for output files

In [3]:
# create output gdb
outputs = '.\\Outputs'
gdb = os.path.join(outputs, "classes_HUI.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "classes_HUI.gdb")
    
scratch = os.path.join(outputs, "scratch_HUI.gdb")
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, "scratch_HUI.gdb")

## Filter Address points (used later)

In [4]:
# get address points without base address point
address_pts_lyr = arcpy.MakeFeatureLayer_management(address_pts,'address_pts_lyr')
query = (''' PtType = 'Residential' ''')
arcpy.SelectLayerByAttribute_management(address_pts_lyr, 'NEW_SELECTION', query)
address_pts_no_base = arcpy.FeatureClassToFeatureClass_conversion(address_pts_lyr, scratch, '_00_address_pts_no_base')

## Prep main parcel layer

In [5]:
# select parcels within modeling area
parcels_layer = arcpy.MakeFeatureLayer_management(parcels, 'parcels') 
arcpy.SelectLayerByLocation_management(parcels_layer, 'HAVE_THEIR_CENTER_IN', taz_shp)
parcels_for_modeling = arcpy.FeatureClassToFeatureClass_conversion(parcels_layer, scratch, '_01_parcels_for_modeling')

# recalc acreage 
arcpy.CalculateField_management(parcels_for_modeling, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

# create the main layer
parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(parcels_for_modeling, 'parcels_for_modeling_lyr')

# house count field is a string for some reason, so this is the fix
query = """ (HOUSE_CNT IS NULL) OR (HOUSE_CNT = ' ') """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
arcpy.CalculateField_management(parcels_for_modeling_layer, "HOUSE_CNT", """0""", "PYTHON3")
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
arcpy.AddField_management(parcels_for_modeling_layer, 'HOUSE_CNT2', 'LONG')
arcpy.CalculateField_management(parcels_for_modeling_layer, "HOUSE_CNT2", """int(!HOUSE_CNT!)""", "PYTHON3")
arcpy.DeleteField_management(parcels_for_modeling_layer,"HOUSE_CNT")

# add second built year field
arcpy.AddField_management(parcels_for_modeling, 'BUILT_YR2', 'SHORT')
arcpy.CalculateField_management(parcels_for_modeling, "BUILT_YR2", """!BUILT_YR!""", "PYTHON3")

<Result '.\\Outputs\\scratch_HUI.gdb\\_01_parcels_for_modeling'>

## More parcel prep

In [6]:
# add a field to indicate parcel type
arcpy.AddField_management(parcels_for_modeling_layer, 'TYPE_WFRC', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'SUBTYPE_WFRC', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'NOTE', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'IS_OUG', 'SHORT')

<Result 'parcels_for_modeling_lyr'>

In [7]:
# count parts and rings
arcpy.AddField_management(parcels_for_modeling_layer, 'PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling_layer, 'RINGS', 'LONG')

fields = ["shape@", 'PARTS', 'RINGS']
with arcpy.da.UpdateCursor(parcels_for_modeling_layer, fields) as cursor:
    for row in cursor:
        shape = row[0]
        parts = shape.partCount
        rings = shape.boundary().partCount   
        row[1] = parts
        row[2] = rings
        cursor.updateRow(row)
        

# delete parcels with empty geometry
query = (""" PARTS =  0""")
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

In [8]:
# get a count of all parcels
count_all = arcpy.GetCount_management(parcels_for_modeling_layer)
print("# initial parcels in modeling area:\n {}".format(count_all))

# initial parcels in modeling area:
 96977


## Load Owned Unit Groupings boundaries (common areas) and summarize parcels within

In [9]:
###############
# Common Areas
###############

common_areas_lyr = arcpy.MakeFeatureLayer_management(common_areas, 'common_areas_lyr') 

# Planned Unit Developments
query = """ SUBTYPE_WFRC IN ('pud') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_pud = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02g_ca_pud')

arcpy.CalculateField_management(ca_pud, field='IS_OUG', expression="1",
                                expression_type="PYTHON3")

# multi_family
query = """ TYPE_WFRC IN ('multi_family') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_multi_family = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02e_ca_multi_family')

arcpy.CalculateField_management(ca_multi_family, field='IS_OUG', expression="1",
                                expression_type="PYTHON3")

<Result '.\\Outputs\\scratch_HUI.gdb\\_02e_ca_multi_family'>

In [10]:
###############
# PUD
###############

tag = "single_family"
tag2 = "pud"

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_pud, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
pud_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_pud_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_pud, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_pud
join_features = pud_centroids
output_features = os.path.join(scratch, "_03a_pud_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total land value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# house count
fieldindex = fieldmappings.findFieldMapIndex('HOUSE_CNT2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'SUM'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square feet
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# floor count
fieldindex = fieldmappings.findFieldMapIndex('FLOORS_CNT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year max
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

arcpy.CalculateField_management(oug_sj, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_pud")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# update year built with max if mode is 0
with arcpy.da.UpdateCursor(oug_sj2, ['BUILT_YR', 'BUILT_YR2']) as cursor:
    for row in cursor:
        if row[0]  is None or row[0] < 1 or row[0] == "":
            row[0] = row[1]
        
        cursor.updateRow(row)

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

2107 "single_family" parcels were selected.
94870 parcels remain...


In [11]:
###############
# multi family
###############

tag = "multi_family"

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_multi_family, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
mf_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_mf_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_multi_family, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_multi_family
join_features = mf_centroids
output_features = os.path.join(scratch, "_03b_mf_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total land value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square feet
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# house count
fieldindex = fieldmappings.findFieldMapIndex('HOUSE_CNT2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'SUM'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# floor count
fieldindex = fieldmappings.findFieldMapIndex('FLOORS_CNT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year max
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_multi_family")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# update year built with max if mode is 0
with arcpy.da.UpdateCursor(oug_sj2, ['BUILT_YR', 'BUILT_YR2']) as cursor:
    for row in cursor:
        if row[0]  is None or row[0] < 1 or row[0] == "":
            row[0] = row[1]
        
        cursor.updateRow(row)

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

8821 "multi_family" parcels were selected.
86049 parcels remain...


## Categorize parcels

In [12]:
################
# single_family
################

query = (''' PROP_CLASS IN ('108- SFR on COMM Zone', '111- SNGL FAM RES', '502-COMM LAND with SFR', 
            '508- Both RES & COMM IMPRVD') OR PARCEL_ID IN ('161300020', '010600021','130340037') ''')
tag="single_family"


# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# these parcels are probably sf residential, but with no building
query = (''' PROP_CLASS IN ('121-OUT BLDGS', '957-RELATED PARCEL') OR PARCEL_ID IN ('130060033','130170033') ''')
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
single_family = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

61784 "single_family" parcels were selected.
24265 parcels remain...


In [13]:
################
# Duplex
################

tag="multi_family"
tag2="duplex"

query = (''' PROP_CLASS IN ('112- DUPLEX') ''')

arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)


# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag2))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1484 "multi_family" parcels were selected.
22781 parcels remain...


In [14]:
################
# Apartment
################

query = (''' PROP_CLASS IN ('113- 3-4 UNITS','114- 5-9 UNITS', '115- 10+ UNITS') OR 
            PARCEL_ID IN ('010600021', '122210002','120840030', '120310022')''')
tag="multi_family"
tag2="apartment"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
mf2_commons = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_02_mf2_commons')

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = mf2_commons
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_apartment")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

827 "multi_family" parcels were selected.
21954 parcels remain...


In [15]:
#################
# mobile home parks
#################

tag = "multi_family"
tag2 = "mobile_home_park"

# use overlay to select mobile home parks parcels
mobile_home_parks = ".\\Inputs\\Mobile_Home_Parks.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=mobile_home_parks,
                                       selection_type='NEW_SELECTION')

query= (""" PROP_CLASS IN ('118- MOBILE HOME') """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
mhp = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_07a_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# recalc acreage 
# arcpy.CalculateGeometryAttributes_management(mhp, [["PARCEL_ACRES", "AREA"]], area_unit='ACRES')
arcpy.CalculateField_management(mhp, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = mhp
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_mobile_home_park")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

30 "multi_family" parcels were selected.
21924 parcels remain...


## Merge the residential parcels back to together and format/process using pandas

In [16]:
# create paths
res_files = ['_02_pud', '_02_single_family','_02_duplex', '_02_mobile_home_park', '_02_multi_family', '_02_apartment']
res_files = [os.path.join(gdb, res_file) for res_file in res_files]

# merge features
rf_merged = arcpy.Merge_management(res_files, os.path.join(scratch, '_10_Housing_Unit_Inventory'), 
                                   add_source='ADD_SOURCE_INFO')

#-----------------------------
# add cities as an attribute
#-----------------------------

cities = r'.\Inputs\Cities.shp' 
target_features = rf_merged
join_features = cities
output_features = os.path.join(scratch, '_04_Housing_Unit_Inventory')

rf_merged = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="HAVE_THEIR_CENTER_IN")

#---------------------------------
# add subcounties as an attribute
#---------------------------------
subcounties = r'.\Inputs\SubCountyArea_2019.shp' 
target_features = rf_merged
join_features = subcounties
output_features = os.path.join(scratch, '_10c_Housing_Unit_Inventory_SubCounty')
rf_merged = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="HAVE_THEIR_CENTER_IN")

# convert to dataframe and export
rf_merged_df = pd.DataFrame.spatial.from_featureclass(rf_merged)

rf_merged_df = rf_merged_df.rename(columns={"ap_count": "ADDR_CNT"})
rf_merged_df = rf_merged_df.rename(columns={"HOUSE_CNT2": "HOUSE_CNT"})
rf_merged_df = rf_merged_df.rename(columns={"NAME": "CITY"})
rf_merged_df = rf_merged_df.rename(columns={"NewSA": "SUBREGION"})
rf_merged_df = rf_merged_df.rename(columns={"parcel_count": "PARCEL_COUNT"})

In [17]:
# calc note column using property class
rf_merged_df.loc[(rf_merged_df['NOTE'].isnull() == True ), 'NOTE'] = rf_merged_df['PROP_CLASS']

# convert unit count columns to int
rf_merged_df['HOUSE_CNT'] = 0
rf_merged_df['HOUSE_CNT'] = rf_merged_df['HOUSE_CNT'].astype(int)
rf_merged_df.loc[(rf_merged_df['ADDR_CNT'].isnull() == True ), 'ADDR_CNT'] = 0
rf_merged_df['ADDR_CNT'] = rf_merged_df['ADDR_CNT'].astype(int)

# create new count field and calculate
rf_merged_df['UNIT_COUNT'] = rf_merged_df['ADDR_CNT']


# fix single family (non-pud)
rf_merged_df.loc[(rf_merged_df['UNIT_COUNT'] == 0) & 
                 (rf_merged_df['TYPE_WFRC'] == 'single_family'), 
                 'UNIT_COUNT'] = 1

# fix duplex
rf_merged_df.loc[(rf_merged_df['UNIT_COUNT'] == 0) & 
                 (rf_merged_df['TYPE_WFRC'] == 'duplex'), 
                 'UNIT_COUNT'] = 2

rf_merged_df.loc[(rf_merged_df['SUBTYPE_WFRC'] == 'duplex'), 'UNIT_COUNT'] = 2

# fix apts
rf_merged_df.loc[(rf_merged_df['UNIT_COUNT'] == 0) & 
                 (rf_merged_df['PROP_CLASS'] == '115- 10+ UNITS'), 
                 'UNIT_COUNT'] = 10

rf_merged_df.loc[(rf_merged_df['UNIT_COUNT'] == 0) & 
                 (rf_merged_df['PROP_CLASS'] == '114- 5-9 UNITS'), 
                 'UNIT_COUNT'] = 5

rf_merged_df.loc[(rf_merged_df['UNIT_COUNT'] == 0) & 
                 (rf_merged_df['PROP_CLASS'] == '113- 3-4 UNITS'), 
                 'UNIT_COUNT'] = 3

# zero out non-ougs
rf_merged_df.loc[(rf_merged_df['IS_OUG'] != 1), 'IS_OUG'] = 0

# calculate the decade
rf_merged_df['BUILT_DECADE'] = 'NA'
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1840) & (rf_merged_df['BUILT_YR'] < 1850), 'BUILT_DECADE'] = "1840's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1850) & (rf_merged_df['BUILT_YR'] < 1860), 'BUILT_DECADE'] = "1850's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1860) & (rf_merged_df['BUILT_YR'] < 1870), 'BUILT_DECADE'] = "1860's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1870) & (rf_merged_df['BUILT_YR'] < 1880), 'BUILT_DECADE'] = "1870's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1880) & (rf_merged_df['BUILT_YR'] < 1890), 'BUILT_DECADE'] = "1880's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1890) & (rf_merged_df['BUILT_YR'] < 1900), 'BUILT_DECADE'] = "1890's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1900) & (rf_merged_df['BUILT_YR'] < 1910), 'BUILT_DECADE'] = "1900's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1910) & (rf_merged_df['BUILT_YR'] < 1920), 'BUILT_DECADE'] = "1910's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1920) & (rf_merged_df['BUILT_YR'] < 1930), 'BUILT_DECADE'] = "1920's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1930) & (rf_merged_df['BUILT_YR'] < 1940), 'BUILT_DECADE'] = "1930's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1940) & (rf_merged_df['BUILT_YR'] < 1950), 'BUILT_DECADE'] = "1940's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1950) & (rf_merged_df['BUILT_YR'] < 1960), 'BUILT_DECADE'] = "1950's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1960) & (rf_merged_df['BUILT_YR'] < 1970), 'BUILT_DECADE'] = "1960's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1970) & (rf_merged_df['BUILT_YR'] < 1980), 'BUILT_DECADE'] = "1970's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1980) & (rf_merged_df['BUILT_YR'] < 1990), 'BUILT_DECADE'] = "1980's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1990) & (rf_merged_df['BUILT_YR'] < 2000), 'BUILT_DECADE'] = "1990's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 2000) & (rf_merged_df['BUILT_YR'] < 2010), 'BUILT_DECADE'] = "2000's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 2010) & (rf_merged_df['BUILT_YR'] < 2020), 'BUILT_DECADE'] = "2010's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 2020) & (rf_merged_df['BUILT_YR'] < 2030), 'BUILT_DECADE'] = "2020's"

# add county
rf_merged_df['COUNTY'] = 'WEBER'

# remove data points with zero units
rf_merged_df = rf_merged_df[~((rf_merged_df['UNIT_COUNT'] == 0) & (rf_merged_df['HOUSE_CNT'] == 0))].copy()

# flag parcels with a large descrepancy between house count and unit count
rf_merged_df['FLAG'] = 0
rf_merged_df.loc[((rf_merged_df['UNIT_COUNT'] / rf_merged_df['ADDR_CNT']) > 1.2) | 
                  ((rf_merged_df['UNIT_COUNT'] / rf_merged_df['ADDR_CNT']) < 0.8), 'FLAG'] = 1

In [18]:
# Final ordering and subsetting of fields
rf_merged_df = rf_merged_df[['OBJECTID','PARCEL_ID','TYPE_WFRC','SUBTYPE_WFRC','NOTE','IS_OUG','CITY','SUBREGION','COUNTY', 
                             'HOUSE_CNT','ADDR_CNT','UNIT_COUNT','PARCEL_COUNT','FLAG','FLOORS_CNT',
                             'PARCEL_ACRES','BLDG_SQFT','TOTAL_MKT_VALUE','BUILT_YR','BUILT_DECADE','SHAPE']].copy()

In [ ]:
# export to feature class
rf_merged_df.spatial.to_featureclass(location=os.path.join(gdb, '_04_weber_housing_unit_inventory'),sanitize_columns=False)

In [ ]:
# export the final table
del rf_merged_df['SHAPE']
rf_merged_df.to_csv(os.path.join(outputs, 'weber_housing_unit_inventory.csv'), index=False)